# Tryout intel-caffe

In [9]:
import os
import sys
from datetime import datetime
RESOURCES_PATH = os.path.join(os.path.abspath(os.path.dirname('.')), '..', 'resources')

In [10]:
env=""

def submit_job(cmd, env=env, name="STDIN"):
    job_id = !echo {env} {cmd} | qsub -l nodes=4:knl -d /home/u2459 -N {name}
    return job_id 

def checkout_job(job_id, name="STDIN", full=False):
    print(job_id[0])
    if not full:
        !qstat
    else:
        !qstat -f {job_id[0].split('.')[0]}
    !cat ~/{name}.o{job_id[0].split('.')[0]}
    !cat ~/{name}.e{job_id[0].split('.')[0]}

In [11]:
!ls {RESOURCES_PATH}

CCS_ResNet_50_solver.prototxt	  ResNet-50-model.caffemodel
CCS_ResNet_50_train_val.prototxt  ResNet_50_solver.prototxt
digits				  ResNet_50_train_val.prototxt
get_resnet50.sh			  ResNet_mean.binaryproto
ResNet-50-deploy.prototxt


## Create solver file

- Define parameters and save as a .prototxt file

In [ ]:
nb_train_images = 550
nb_val_images = 250

In [ ]:
solver_params = {
    # Specify the network
    'net': ''
    # Specify nb epochs
    'max_iter: 10000': ''
}

now = datetime.now()
solver_filename = os.path.join(RESOURCES_PATH, 'solver_%s.prototxt' % (str(now.strftime("%Y-%m-%d-%H-%M"))))


In [ ]:
assert not os.path.exists(solver_filename), "Solver configuration file already exists"

with open(solver_filename, 'w') as w:
    w

In [19]:
!ls /home/u2459/DIGITS/digits/

config		  job.py	     static		   test_version.py
dataset		  job.pyc	     status.py		   test_views.py
device_query.py   jobs		     status.pyc		   tools
device_query.pyc  log.py	     store		   utils
digits.log	  log.pyc	     task.py		   version.py
download_data	  __main__.py	     task.pyc		   version.pyc
extensions	  model		     templates		   views.py
frameworks	  pretrained_model   test_device_query.py  views.pyc
inference	  scheduler.py	     test_scheduler.py	   webapp.py
__init__.py	  scheduler.pyc      test_status.py	   webapp.pyc
__init__.pyc	  standard-networks  test_utils.py


In [20]:
!cat /home/u2459/DIGITS/digits/__main__.py

# Copyright (c) 2014-2017, NVIDIA CORPORATION.  All rights reserved.

import argparse
import os.path
import sys


# Update PATH to include the local DIGITS directory
PARENT_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
found_parent_dir = False
for p in sys.path:
    if os.path.abspath(p) == PARENT_DIR:
        found_parent_dir = True
        break
if not found_parent_dir:
    sys.path.insert(0, PARENT_DIR)


def main():
    parser = argparse.ArgumentParser(description='DIGITS server')
    parser.add_argument(
        '-p', '--port',
        type=int,
        default=5000,
        help='Port to run app on (default 5000)'
    )
    parser.add_argument(
        '-d', '--debug',
        action='store_true',
        help=('Run the application in debug mode (reloads when the source '
              'changes and gives more detailed error messages)')
    )
    parser.add_argument(
        '--version',
        action='store_true',
        help='Print the version number and ex

In [18]:
!cat /home/u2459/start_digits.sh

#!/bin/sh

source ~/env.local

export PYTHONPATH=$PYTHONPATH:~/digits_dependencies/lib/python2.7/site-packages/

export CAFFE_ROOT=/opt/caffe-master/

## START DIGITS SERVER at localhost:5000
pushd DIGITS

set -e
python2 -m digits $@

popd


## Start training

In [43]:
caffe_train_cmd = [
    "/opt/caffe-master/build/tools/caffe",
    "train", 
    "--solver="+solver_filename,
    "--weights="+os.path.join(RESOURCES_PATH, 'ResNet-50-model.caffemodel'),
]
caffe_train_cmd = ' '.join(caffe_train_cmd)

caffe_help_cmd = [
    "/opt/caffe-master/build/tools/caffe",
    "--help", 
]
caffe_help_cmd = ' '.join(caffe_help_cmd)

In [83]:
job_id = submit_job(caffe_train_cmd, name='caffe_train')
# job_id = submit_job(caffe_help_cmd)

In [120]:
checkout_job(job_id, full=False)

3535.c001
Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
3535.c001                  STDIN            u2459           12:27:20 R batch          
cat: /home/u2459/STDIN.o3535: No such file or directory
cat: /home/u2459/STDIN.e3535: No such file or directory
